# Design of Experiments

A Design of Experiments (DoE) is a convenient way to run multiple simulations.

## Linear DoE

The linear DoE generates an hypercube of the inputs, using a linspace.

Here we illustrate how it works with a design DoE.

In [1]:
from cosapp.drivers import NonLinearSolver, LinearDoE, RunSingleCase
from cosapp.recorders import DataFrameRecorder

from cpu.systems import CPUSystem

#### Create a new CPU system 

In [2]:
cpu = CPUSystem("cpu")

#### Add a DoE driver to run 31 designs from 30 to 60°C as air temperature range

In [3]:
# add DoE driver
doe = cpu.add_driver(LinearDoE("doe"))
doe.add_recorder(DataFrameRecorder(includes=["*"]))
doe.add_input_var(
    {
        "fan.T_air": {"lower": 20.0, "upper": 40.0, "count": 21},
        "T_cpu": {"lower": 70.0, "upper": 80.0, "count": 21},
    }
)

#### Add complementary drivers for design

In [4]:
# add a NLS solver
design = doe.add_child(NonLinearSolver("solver", tol=1e-6))
design.extend(cpu.design_methods["exchanger_surface"])

# add a driver to set initial values
runner = design.add_driver(RunSingleCase("runner"))
runner.set_values({"cpu.usage": 100.0})

#### Run and get results

In [5]:
cpu.run_drivers()
results = doe.recorder.data

#### Plot chart of multiple designs 

In [6]:
import plotly.graph_objects as go

title = "Heat exchanger surface computed through a DoE"
fig = go.Figure(
    data=go.Scatter(
        x=results["fan.T_air"],
        y=results["T_cpu"],
        mode="markers",
        marker=dict(
            color=results["exchanger.surface"],
            colorbar={"title": "Exchanger surface (m<sup>2</sup>)"},
        ),
    )
)
fig.update_layout(
    xaxis_title="Air temperature (°C)", yaxis_title="CPU temperature (°C)", title=title
)
fig.show()

In [7]:
from dash import Dash, dcc, html, Input, Output
import numpy as np

app = Dash(__name__)


app.layout = html.Div(
    [
        html.H4("Design of heat exchanger surface according to air and CPU temperatures"),
        dcc.Graph(id="graph"),
        html.P("CPU temperature (°C):"),
        dcc.Slider(
            id="tcpu",
            min=70.0,
            max=80.0,
            step=0.5,
            value=75,
            marks={float(val): str(val) for val in np.unique(results["T_cpu"])},
        ),
    ]
)


@app.callback(Output("graph", "figure"), Input("tcpu", "value"))
def display_color(tcpu):
    data = results.query(f"T_cpu == {tcpu}")
    fig = go.Figure(
        data=go.Scatter(x=data["fan.T_air"], y=data["exchanger.surface"], mode="markers")
    )
    fig.update_layout(
        xaxis_title="Air temperature (°C)", yaxis_title="Heat exchanger surface (m<sup>2</sup>)"
    )
    return fig


app.run()